In [ ]:
import pandas as pd;
import nltk
import re
import string
import numpy as np
string.punctuation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score,f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')




import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Information_retrieval/A2_Q2.csv", encoding = "ISO-8859-1")
df.head(10)

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business
5,1582,howard truanted to play snooker conservative...,politics
6,651,wales silent on grand slam talk rhys williams ...,sport
7,1797,french honour for director parker british film...,entertainment
8,2034,car giant hit by mercedes slump a slump in pro...,business
9,1866,fockers fuel festive film chart comedy meet th...,entertainment


In [ ]:
# Checking NULL values of each column

df.isnull().sum()

ArticleId    0
Text         0
Category     0
dtype: int64

In [ ]:
# Dropping the column

df = df.drop(['ArticleId'], axis=1)

In [ ]:
# Removing punctuation

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

# Storing the puntuation free text

df['Text']= df['Text'].apply(lambda x:remove_punctuation(x))
df.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in 168m payout eighteen former en...,business


In [ ]:
# Converting the all the text in the lowercase.

df['Category']= df['Category'].apply(lambda x: x.lower())

df['Text']= df['Text'].apply(lambda x: x.lower())

df.head()


,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in 168m payout eighteen former en...,business


In [ ]:
# Preprocessing the dataset.

stop_words = set(stopwords.words('english'))

# Initialize stemmer

stemmer = PorterStemmer()

def preprocessing_dataset(text):

    # Remove whitespace
    text=  re.sub(' +', ' ', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    processed_tokens = [token for token in tokens if token not in stop_words]

    # Perform stemming
    stemmed_tokens = [stemmer.stem(token) for token in processed_tokens]
    return ' '.join(stemmed_tokens)

In [ ]:
# Preprocessing 

updated_text = [preprocessing_dataset(text) for text in df['Text']]

In [ ]:
# term document 

term_doc = CountVectorizer().fit_transform(updated_text)

In [ ]:
# TF-IDF 

tf_idf_matrix = TfidfTransformer().fit_transform(term_doc).toarray()


In [ ]:
# Calculating ICF 
num_docs = len(df)
icf = np.log(num_docs / np.count_nonzero(tf_idf_matrix, axis=0))

In [ ]:
# Converting tfidf and icf to sparse matrices

tfidf_matrix = csr_matrix(tf_idf_matrix)
icf = csr_matrix(icf)

In [ ]:
# Compute the TF-ICF
tf_icf = tfidf_matrix.multiply(icf)

In [ ]:
print(tf_icf)

  (0, 19606)	0.10889205198430775
  (0, 19573)	0.008007005010754938
  (0, 19441)	2.5116048084301688
  (0, 19355)	0.43127312668158546
  (0, 19230)	0.5607384552093957
  (0, 19109)	0.03217036938024452
  (0, 19105)	0.09122074155620755
  (0, 19024)	0.06681431475818761
  (0, 18976)	0.20449198889429906
  (0, 18778)	0.35359487977417275
  (0, 18700)	0.09989043993963045
  (0, 18641)	0.027270847768020513
  (0, 18624)	0.020321011806398313
  (0, 18330)	0.39967044232612003
  (0, 18308)	0.2880634540053271
  (0, 18040)	0.2442351105941548
  (0, 17840)	0.03020974571860851
  (0, 17523)	0.12169570325070507
  (0, 17510)	0.18105378649676682
  (0, 17483)	0.14378230336963452
  (0, 17135)	0.4573280121882653
  (0, 16993)	0.10384549838485468
  (0, 16808)	0.060552227058777976
  (0, 15977)	0.1497012419931611
  (0, 15871)	0.08718029915616202
  :	:
  (1489, 6914)	0.11324391785101001
  (1489, 6685)	0.13735368825701377
  (1489, 6093)	0.10932576668133451
  (1489, 5744)	1.7006642448107794
  (1489, 5708)	0.165251487214855

In [ ]:
#Spliting training data and test data into 70:30 ratio

x_train,x_test,y_train,y_test = train_test_split(tf_icf,df['Category'],test_size=0.3,random_state=10)

In [ ]:
# Naive Bayes classifier
classifier = MultinomialNB()

# Fit the classifier on the training data
classifier.fit(x_train, y_train)

# Predict the categories of the testing data
predict_y = classifier.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, predict_y)

# Accuracy score
print('Accuracy:', accuracy)

Accuracy: 0.9731543624161074


In [ ]:
# Confusion matrix
confusion_matrix = confusion_matrix(y_test, predict_y)
print(confusion_matrix)

[[ 87   0   2   0   2]
 [  1  82   2   0   2]
 [  1   1  72   0   0]
 [  0   0   0 118   0]
 [  0   1   0   0  76]]


In [ ]:
precision = precision_score(y_test, predict_y, average='macro')
recall = recall_score(y_test, predict_y, average='macro')
f1 = f1_score(y_test, predict_y, average='macro')


print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 score: {f1:.4f}')

Precision: 0.9702
Recall: 0.9717
F1 score: 0.9708


In [ ]:
#Spliting training data and test data into 60:40 ratio

x_train1,x_test1,y_train1,y_test1 = train_test_split(tf_icf,df['Category'],test_size=0.4,random_state=36)

In [ ]:
# Naive Bayes classifier
classifier1 = MultinomialNB()

# Fit the classifier on the training data
classifier1.fit(x_train1, y_train1)

# Predict the categories of the testing data
predict_y1 = classifier1.predict(x_test1)

In [ ]:
accuracy1 = accuracy_score(y_test1, predict_y1)

# Accuracy score
print('Accuracy:', accuracy1)

Accuracy: 0.9630872483221476


In [ ]:
# Confusion matrix
confusion_matrix1 = metrics.confusion_matrix(y_test1, predict_y1)
print(confusion_matrix1)

[[128   0   4   1   1]
 [  2 126   0   0   6]
 [  2   0 101   0   1]
 [  0   0   0 129   0]
 [  1   3   1   0  90]]


In [ ]:


precision = precision_score(y_test, predict_y, average='macro')
recall = recall_score(y_test, predict_y, average='macro')
f1 = f1_score(y_test, predict_y, average='macro')


print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 score: {f1:.4f}')

Precision: 0.9702
Recall: 0.9717
F1 score: 0.9708


In [ ]:
#Spliting training data and test data into 50:50 ratio

x_train2,x_test2,y_train2,y_test2 = train_test_split(tf_icf,df['Category'],test_size=0.5,random_state=36)

In [ ]:
# Naive Bayes classifier
classifier2 = MultinomialNB()

# Fit the classifier on the training data
classifier2.fit(x_train2, y_train2)

# Predict the categories of the testing data
predict_y2 = classifier2.predict(x_test2)

In [ ]:
accuracy2 = accuracy_score(y_test2, predict_y2)

# Accuracy score
print('Accuracy:', accuracy2)

Accuracy: 0.9624161073825503


In [ ]:
# Precision

precision2 = precision_score(y_test2,predict_y2,average='macro')
print('Precision: %f' % precision)

# recall: tp / (tp + fn)

recall = recall_score(y_test2, predict_y2,average='macro')
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)

f1 = f1_score(y_test2, predict_y2,average='macro')
print('F1 score: %f' % f1)

Precision: 0.970217
Recall: 0.961962
F1 score: 0.961254


In [ ]:
# Confusion matrix
confusion_matrix2 = metrics.confusion_matrix(y_test2, predict_y2)
print(confusion_matrix2)

[[162   0   3   1   2]
 [  2 147   3   1   5]
 [  3   2 123   1   1]
 [  0   0   0 160   0]
 [  1   1   2   0 125]]


In [ ]:
#Spliting training data and test data into 20:80 ratio

x_train3,x_test3,y_train3,y_test3 = train_test_split(tf_icf,df['Category'],test_size=0.8,random_state=30)

In [ ]:
print(x_train3.shape)
print(x_test3.shape)
print(y_train3.shape)
print(y_test3.shape)

(298, 19695)
(1192, 19695)
(298,)
(1192,)


In [ ]:
# Naive Bayes classifier
classifier3 = MultinomialNB()

# Fit the classifier on the training data
classifier3.fit(x_train3, y_train3)

# Predict the categories of the testing data
predict_y3 = classifier3.predict(x_test3)

In [ ]:
accuracy3 = accuracy_score(y_test3, predict_y3)

# Accuracy score
print('Accuracy:', accuracy3)

Accuracy: 0.9588926174496645


In [ ]:
# Precision

precision3 = precision_score(y_test3,predict_y3,average='macro')
print('Precision: %f' % precision3)

# recall: tp / (tp + fn)

recall3 = recall_score(y_test3, predict_y3,average='macro')
print('Recall: %f' % recall)

# f1: 2 tp / (2 tp + fp + fn)

f1 = f1_score(y_test3, predict_y3,average='macro')
print('F1 score: %f' % f1)

# precision = precision_score(y_test3, predict_y3, average='macro')
# recall = recall_score(y_test3, predict_y3, average='macro')
# f1 = f1_score(y_test3, predict_y3, average='macro')


# print(f'Precision: {precision:.4f}')
# print(f'Recall: {recall:.4f}')
# print(f'F1 score: {f1:.4f}')

Precision: 0.957015
Recall: 0.961962
F1 score: 0.957417


In [ ]:
# Confusion matrix
confusion_matrix3 = metrics.confusion_matrix(y_test3, predict_y3)
print(confusion_matrix3)

[[252   1   5   1   9]
 [  0 211   4   0   2]
 [  8   1 207   1   4]
 [  1   0   1 275   0]
 [  2   5   4   0 198]]


In [ ]:
# # Loop over the unique categories in the training set

# for category in y_train.unique():
#     index = y_train[y_train == category].index
#     tf_icf_avg = tf_icf[index].mean(axis=0)
#     tf_icf_avg = np.squeeze(np.asarray(tf_icf_avg))
#     feature_names = TfidfTransformer().get_feature_names_out()
#     for feature_idx in np.argsort(tf_icf_avg)[-10:]:
#         feature_name = feature_names[feature_idx]
#         feature_tficf = tf_icf_avg[feature_idx]
#         print(f'{category}: {feature_name} - {feature_tficf:.4f}')

In [ ]:
df3 = pd.read_csv("/content/drive/MyDrive/Information_retrieval/A2_Q2.csv", encoding = "ISO-8859-1")

In [ ]:
# Dropping the column

df3 = df3.drop(['ArticleId'], axis=1)

In [ ]:
# Removing punctuation

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

# Storing the puntuation free text

df3['Text']= df['Text'].apply(lambda x:remove_punctuation(x))
df3.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in 168m payout eighteen former en...,business


In [ ]:
# Converting the all the text in the lowercase.

df3['Category']= df3['Category'].apply(lambda x: x.lower())

df3['Text']= df3['Text'].apply(lambda x: x.lower())

df3.head()

,Text,Category
0,worldcom exboss launches defence lawyers defen...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in 168m payout eighteen former en...,business


In [ ]:
# Preprocessing the dataset.

stop_words = set(stopwords.words('english'))

# Initialize the WordNetLemmatizer
lemma = WordNetLemmatizer()

def preprocessing_dataset(text):

    # Remove whitespace
    text=  re.sub(' +', ' ', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Removing stopwords
    stop_words = set(stopwords.words('english'))
    processed_tokens = [token for token in tokens if token not in stop_words]

   # Lemmatize the words
    words = [lemma.lemmatize(word) for word in processed_tokens]

    return words

df3['Text'] = df3['Text'].apply(preprocessing_dataset)

In [ ]:
range = (1,2)

# Convert the list of words back to a string for the TfidfVectorizer

df3['Text'] = df3['Text'].apply(' '.join)
cnt_vect2 = TfidfVectorizer(ngram_range = range)

# Fit and transform the vectorizer on the 'Text' column
tfidf2 = cnt_vect2.fit_transform(df3['Text'])

# Convert the sparse matrix to a dense matrix
tfidf2 = tfidf2.toarray()

# Create a DataFrame from the dense matrix
tfidf_df2 = pd.DataFrame(tfidf2, columns=cnt_vect2.get_feature_names_out())

# Add the 'Category' column to the DataFrame
tfidf_df2['Category'] = df3['Category']

# Group the DataFrame by category and calculate the mean of each feature
tficf_df2 = tfidf_df2.groupby('Category').mean()

In [ ]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(tfidf_df2.drop('Category', axis=1), tfidf_df2['Category'], test_size=0.3, random_state=42)

In [ ]:
# Naive Bayes classifier
classifier4 = MultinomialNB()

# Fit the classifier on the training data
classifier4.fit(X_train4, y_train4)

# Predict the categories of the testing data
predict_y4 = classifier4.predict(X_test4)

In [ ]:
accuracy4 = accuracy_score(y_test4, predict_y4)

# Accuracy score
print('Accuracy:', accuracy4)

Accuracy: 0.959731543624161


In [ ]:
# Precision

precision4 = precision_score(y_test4,predict_y4,average='macro')
print('Precision: %f' % precision4)

# recall: tp / (tp + fn)

recall4 = recall_score(y_test4, predict_y4,average='macro')
print('Recall: %f' % recall4)

# f1: 2 tp / (2 tp + fp + fn)

f1 = f1_score(y_test4, predict_y4,average='macro')
print('F1 score: %f' % f1)


Precision: 0.961155
Recall: 0.957556
F1 score: 0.958878


In [ ]:
# Confusion matrix
confusion_matrix4 = metrics.confusion_matrix(y_test4, predict_y4)
print(confusion_matrix4)

[[104   0   2   0   2]
 [  2  73   0   4   0]
 [  2   0  81   1   2]
 [  0   0   0 101   0]
 [  0   0   2   1  70]]
